In [5]:
%reset_selective -f b

# Este comando le indica a Python que las figuras se deben generar dentro de la misma Notebook, no en una ventana
## Solo para notebooks.
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import colors
from sklearn.svm import SVC

import numpy as np
import glob
import pyart
import scipy.io as sio
import re


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [6]:
def read_files(day,init_time):
    day = str(day)
    init_time = str(init_time)
    path_user_radar = './febdBZ/'+ day + '/'
    FileList_radar = np.sort(glob.glob(path_user_radar+'*.nc'))
    
    file2read_radar=0
    for i in range(len(FileList_radar)):
        file2read_radar_time = FileList_radar[i].split('_')[1][:4]
        if (file2read_radar_time == init_time):
            file2read_radar = FileList_radar[i]
    
    return()
    

In [7]:
read_files(20170205,'0950')

()

In [8]:
def get_volumes(day, init_time, bins=240,diff=30):
    diff=diff/10
    day = str(day)
    init_time = str(init_time)
    path_user_radar = './febdBZ/'+ day + '/'
    FileList_radar = np.sort(glob.glob(path_user_radar+'*.nc'))
    proof = 0
    file2read_radar=0
    for i in range(len(FileList_radar)):
        file2read_radar_time = FileList_radar[i].split('_')[1][:4]
        if (file2read_radar_time == init_time):
            file2read_radar = FileList_radar[i]
            print(day)
            proof= int(i+diff)
            
        
    file2follow = FileList_radar[proof]   
               
    #iTime = file2read_radar.split('_')[1]
    #fTime = file2read_radar.split('_')[4]
    #time = [iTime,fTime]
    #iTime2 = file2follow.split('_')[1]
    #fTime2 = file2follow.split('_')[4]
    #time2 = [iTime2,fTime2]
    radar = pyart.io.read(file2read_radar)
    radar2 = pyart.io.read(file2follow)
    
    gatefilter = pyart.filters.GateFilter(radar)
    
    gatefilter.exclude_transition()
    gatefilter.exclude_masked('dBZ')
    
    grid = pyart.map.grid_from_radars((radar,), gatefilters=(gatefilter, ),grid_shape=(1, bins, bins),
    grid_limits=((0, 0), (-bins*1000, bins*1000), (-bins*1000, bins*1000)),fields=['dBZ'])
    
    gatefilter2 = pyart.filters.GateFilter(radar2)
    
    gatefilter2.exclude_transition()
    gatefilter2.exclude_masked('dBZ')
    
    grid2 = pyart.map.grid_from_radars((radar2,), gatefilters=(gatefilter2, ),grid_shape=(1, bins, bins),
    grid_limits=((0, 0), (-bins*1000, bins*1000), (-bins*1000, bins*1000)),fields=['dBZ'])
    #print('grids ready')
    
    
    #return (grid,grid2,time, time2)
    return (grid,grid2)

In [9]:
def get_labels(lines,threshold=35):
    samples = []
    for i in range(len(lines)):
        (fecha,hora)=(lines[i].strip().split("\t")[0],lines[i].strip().split("\t")[1])
        samples.append(get_volumes(fecha,hora)[1])
    
    labels = []
    for i in range(len(lines)):
        grid = samples[i].fields['dBZ']['data'][0]
        for x in range(240):
            for y in range(240):
                if grid[x,y]<35:
                    grid[x,y]=0
                else:
                    grid[x,y]=1
                
        labels.append(grid)
        print('Labels ready')
    return(labels)

In [ ]:
with open('dates.txt') as f:
    datos_rayo = f.readlines()


In [ ]:
etiquetas = get_labels(datos_rayo)


20170202
20170202
20170202
20170202
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170203
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170204
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170208
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
20170209
2

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(etiquetas.fields['dBZ']['data'][0], origin='lower')
plt.show()

In [ ]:
def get_rays(day,init_time):
    folder=day[:6]
    dia=day[2:]
    path_user_rayos = './rayos/'+ folder + '/' + dia
    # Con esto levanto todos los datos de rayos país ese día
    with open(path_user_rayos+'.txt') as f:
        lights = f.readlines()

In [ ]:
lat_grid = grid.get_point_longitude_latitude(level=0, edges=False)[1]
lon_grid = grid.get_point_longitude_latitude(level=0, edges=False)[0]
extLat   = [np.amin(lat_grid),np.amax(lat_grid)]
difLat   = bins/(extLat[1]-extLat[0])
extLon   = [np.amin(lon_grid),np.amax(lon_grid)]
difLon   = bins/(extLon[1]-extLon[0])